# Example 3: Cross-Validation Fold Analysis

**Phase 4 Features Showcased:**
- ✅ Panel Details Modal (click for full-size analysis)
- ✅ Modal Navigation (Next/Previous through folds)
- ✅ Responsive Design (mobile/tablet testing)
- ✅ Search (find specific time periods/folds)
- ✅ Help & Documentation (in-app help for CV metrics)
- ✅ Error Handling (empty states for problematic folds)

## Use Case

Examine model performance across 10 cross-validation folds and 3 different time windows. Each panel shows fold-specific residual analysis to identify temporal patterns or problematic validation periods.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from trelliscope import Display
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

## 1. Generate Cross-Validation Results

In [ ]:
# Import required libraries at the top
from scipy import stats
from pandas.plotting import autocorrelation_plot

def create_residual_analysis_plot(residuals, fold_id, model, train_period, test_period, rmse, r2):
    """Create comprehensive residual analysis visualization."""
    fig = plt.figure(figsize=(14, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    # Residuals over time
    ax1 = fig.add_subplot(gs[0, :])
    t = np.arange(len(residuals))
    ax1.scatter(t, residuals, alpha=0.5, s=20, color='#3498DB')
    ax1.axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.7)
    ax1.axhline(y=1.96*np.std(residuals), color='orange', linestyle=':', linewidth=1.5, alpha=0.7, label='±1.96σ')
    ax1.axhline(y=-1.96*np.std(residuals), color='orange', linestyle=':', linewidth=1.5, alpha=0.7)
    ax1.set_xlabel('Time Index', fontsize=11)
    ax1.set_ylabel('Residuals', fontsize=11)
    ax1.set_title(f'{model} - {fold_id}\nTest Period: {test_period}', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.25)
    
    # Histogram
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.hist(residuals, bins=30, color='#9B59B6', alpha=0.7, edgecolor='black')
    ax2.axvline(x=0, color='red', linestyle='--', linewidth=2)
    ax2.set_xlabel('Residual Value', fontsize=10)
    ax2.set_ylabel('Frequency', fontsize=10)
    ax2.set_title('Distribution', fontsize=11, fontweight='bold')
    ax2.grid(True, alpha=0.25)
    
    # Q-Q Plot
    ax3 = fig.add_subplot(gs[1, 1])
    stats.probplot(residuals, dist="norm", plot=ax3)
    ax3.set_title('Q-Q Plot (Normality Check)', fontsize=11, fontweight='bold')
    ax3.grid(True, alpha=0.25)
    
    # Autocorrelation
    ax4 = fig.add_subplot(gs[2, 0])
    autocorrelation_plot(pd.Series(residuals), ax=ax4, color='#E74C3C')
    ax4.set_title('Autocorrelation', fontsize=11, fontweight='bold')
    ax4.set_xlabel('Lag', fontsize=10)
    ax4.set_ylabel('Correlation', fontsize=10)
    
    # Metrics summary
    ax5 = fig.add_subplot(gs[2, 1])
    ax5.axis('off')
    metrics_text = f"""
    Performance Metrics
    
    RMSE: {rmse:.4f}
    R² Score: {r2:.4f}
    Mean Residual: {np.mean(residuals):.4f}
    Std Residual: {np.std(residuals):.4f}
    
    Train Period: {train_period}
    Test Period: {test_period}
    """
    ax5.text(0.1, 0.5, metrics_text, fontsize=11, family='monospace',
             verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    return fig

# Generate CV fold results
n_folds = 10
models = ['RandomForest', 'XGBoost', 'LightGBM']
n_test_points = 100

print(f"Generating CV fold analysis for {n_folds} folds × {len(models)} models = {n_folds * len(models)} panels...\n")

In [ ]:
data_rows = []
panel_count = 0

base_date = datetime(2020, 1, 1)

for model in models:
    for fold in range(1, n_folds + 1):
        # Generate residuals with some temporal patterns
        trend = np.linspace(0, 0.2 * (fold % 3 - 1), n_test_points)  # Some folds have trend in residuals
        seasonal = 0.5 * np.sin(2 * np.pi * np.arange(n_test_points) / 20)
        noise = np.random.normal(0, 1 + 0.1 * fold, n_test_points)  # Later folds slightly more variable
        residuals = trend + seasonal + noise
        
        # Calculate metrics
        rmse = np.sqrt(np.mean(residuals**2))
        # Simulate predictions to get R²
        y_true = 50 + np.random.normal(0, 5, n_test_points)
        y_pred = y_true - residuals
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y_true - np.mean(y_true))**2)
        r2_score = 1 - (ss_res / ss_tot)
        
        # Time periods
        train_start = base_date + timedelta(days=(fold-1) * 365)
        train_end = base_date + timedelta(days=fold * 365 - 31)
        test_start = train_end + timedelta(days=1)
        test_end = test_start + timedelta(days=30)
        
        train_period = f"{train_start.strftime('%Y-%m')} to {train_end.strftime('%Y-%m')}"
        test_period = f"{test_start.strftime('%Y-%m')} to {test_end.strftime('%Y-%m')}"
        
        # Create visualization
        fig = create_residual_analysis_plot(
            residuals, f"Fold_{fold}", model, train_period, test_period, rmse, r2_score
        )
        
        data_rows.append({
            'panel': fig,
            'fold_id': f"Fold_{fold}",
            'fold_number': fold,
            'model': model,
            'train_period': train_period,
            'test_period': test_period,
            'rmse': rmse,
            'r2_score': r2_score,
            'residual_std': np.std(residuals),
            'prediction_bias': np.mean(residuals),
            'test_start_date': test_start
        })
        
        panel_count += 1
        print(f"  Generated {panel_count}/{n_folds * len(models)} panels...")
        
        plt.close(fig)

print(f"\n✓ Generated {panel_count} CV fold visualizations")

## 2. Create Trelliscope Display

In [ ]:
df = pd.DataFrame(data_rows)

print(f"\nDataFrame shape: {df.shape}")
print(f"\nFold RMSE summary:")
print(df.groupby('model')['rmse'].agg(['mean', 'std', 'min', 'max']))

In [ ]:
display = (
    Display(df, name="cv_fold_analysis", description="Cross-validation fold analysis across 10 folds for 3 models with comprehensive residual diagnostics")
    .set_panel_column("panel")
    .infer_metas()
    .set_default_layout(ncol=3, nrow=2)
    .set_default_labels(["fold_id", "model", "rmse", "r2_score"])
)


## 3. Launch Interactive Viewer

In [ ]:
from trelliscope.dash_viewer import create_dash_app

app = create_dash_app(display, force_write=True)

print("\n" + "="*70)
print("🚀 LAUNCHING CV FOLD ANALYSIS VIEWER")
print("="*70)
print(f"\n📊 Display: {display.name}")
print(f"📈 Total Folds: {len(df)}")
print(f"\n💡 TIP: Click any panel to open full-size modal")
print(f"         Use Next/Previous to navigate through folds")
print(f"         Perfect for detailed residual analysis!")
print("\n🌐 Opening browser on http://localhost:8053...\n")

app.run(debug=False, host='127.0.0.1', port=8053)

## 4. Feature Testing Guide

### ✅ Panel Details Modal (PRIMARY FOCUS)

**Try This**:
1. **Open Modal**:
   - Click any CV fold panel
   - Observe modal opens with full-size visualization
   - All 6 subplots should be clearly visible

2. **Modal Navigation** (Next/Previous buttons):
   - Click "Next" button → see next fold
   - Click "Previous" button → go back
   - Navigate through all 10 folds for a model
   - Observe metadata updates in modal

3. **Keyboard Navigation in Modal**:
   - Press **→** (right arrow) → next fold
   - Press **←** (left arrow) → previous fold
   - Press **Esc** → close modal

4. **Compare Folds**:
   - Filter: model = "XGBoost"
   - Click Fold_1 panel → open modal
   - Use Next button to step through Fold_1 → Fold_10
   - Look for patterns in residuals

**Expected**: Modal shows full detail, navigation works, Esc closes

---

### ✅ Responsive Design (Feature 7)

**Try This**:
1. **Desktop View** (>991px):
   - Verify sidebar on left
   - Grid shows 3 columns
   - All controls accessible

2. **Tablet View** (768-991px):
   - Open DevTools (F12)
   - Toggle device toolbar
   - Select iPad (768px width)
   - Verify:
     - Sidebar becomes collapsible
     - Controls stack vertically
     - Touch targets are 44px+
     - Panels remain readable

3. **Mobile View** (480-767px):
   - Select iPhone (375px width)
   - Verify:
     - Grid becomes 1 column (full-width panels)
     - All buttons are touch-friendly (44px height)
     - Input font-size is 16px (no iOS zoom)
     - No horizontal scrolling

4. **Modal on Mobile**:
   - In mobile view, click a panel
   - Verify modal takes near-full screen
   - Verify Next/Previous buttons are large enough
   - Verify can close modal easily

**Expected**: Layout adapts smoothly, all features usable on mobile

---

### ✅ Search for Specific Folds/Periods

**Try This**:
1. Search **"Fold_5"** → see Fold 5 across all models
2. Search **"XGBoost"** → see only XGBoost folds
3. Search **"2022"** → see folds with 2022 test periods
4. Press **"/"** → focus search
5. Press **Esc** → clear search

**Expected**: Search finds text in fold_id, model, test_period

---

### ✅ Error Handling & Empty States

**Try This**:
1. **Problematic Folds Filter**:
   - Filter: rmse > 10.0 (impossible threshold)
   - Observe empty state appears:
     - Icon (inbox)
     - "No Results Found" title
     - "No panels match current filters" message
     - "Reset Filters" button

2. **Reset from Empty State**:
   - Click "Reset Filters" button
   - Verify filters clear
   - Verify panels return

3. **Toast Notifications**:
   - Save a view → see success toast (top-right)
   - Toast auto-dismisses after 3 seconds
   - Or click X to dismiss immediately

**Expected**: Empty state is helpful, toast feedback is non-intrusive

---

### ✅ Help & Documentation (Feature 8)

**Try This**:
1. Click **"?" (Help) button** in top-right
2. Scroll to **"Panel Details"** section
3. Read about modal navigation
4. Scroll to **"Filters"** section
5. Learn about filtering CV metrics
6. Close help modal
7. Apply what you learned

**Expected**: Help explains CV analysis features clearly

---

## 5. Analysis Workflow Example

### Identify Problematic Folds

1. **Find High-Error Folds**:
   - Sort: rmse (descending)
   - Click top panel → open modal
   - Examine residual plots for patterns
   - Use Next to compare to other high-error folds

2. **Check for Temporal Patterns**:
   - Filter: prediction_bias > 0.5 (positive bias)
   - Check test_period dates
   - Look for seasonal effects

3. **Model Comparison**:
   - Filter: fold_id = "Fold_3"
   - See Fold 3 across all 3 models
   - Click each to compare residual patterns
   - Which model handles this period best?

4. **Save Investigation View**:
   - Filter: rmse > 2.0
   - Sort: rmse descending
   - Save as "Investigate These Folds"
   - Can return to this later

---

## Summary

This example demonstrates:
- ✅ Panel details modal with full-size diagnostics
- ✅ Modal navigation (Next/Previous through folds)
- ✅ Responsive design (mobile/tablet/desktop)
- ✅ Search across fold IDs and time periods
- ✅ Error handling with empty states
- ✅ In-app help for CV metrics

**Next**: Try Example 5 (Multi-Series at Scale) for 100+ panel performance!